#### Материал для экспериментов

In [ ]:
import pandas as pd

df = pd.read_csv("wiki_prep2.csv")
corpus = df[['en', 'ru']]
corpus.head()

## Стандартная модель LDA на отдельных подкорпусах
1. Выбор количества тем (сранивнить связность тем для каждого подкорпуса)
2. Построение моделей для отдельных языков
3. Сравнить полученные темы

In [ ]:
import pandas as pd

# ff = pd.read_csv("eng2.csv")
stop_en = ['also', 'centuri', 'year', 'new', 'one', 'first', 'two']
stop_ru = []
data_en = [[x for x in str(s).split() if len(x) > 2 and x not in stop_en] for s in corpus.en]
# data_en = [x.
data_ru = [[x for x in str(s).split() if len(x) > 2 and x not in stop_ru] for s in corpus.ru]
print(len(data_en), len(data_ru))

In [ ]:
df = pd.read_csv("c:\asus\desktop\wiki_prep2.csv")
stop_en = ['also', 'century', 'year', 'new', 'one', 'first', 'two']
stop_ru = 'первый два один век'.split()
data_en = [[x for x in str(s).split() if len(x) > 2 and x not in stop_en] for s in corpus.en]
data_ru = [[x for x in str(s).split() if len(x) > 2 and x not in stop_ru] for s in corpus.ru]

In [ ]:
from gensim import corpora, models
from gensim.models import LdaModel, LdaMulticore
from gensim.models.coherencemodel import CoherenceModel
from gensim.corpora import Dictionary

import matplotlib.pyplot as plt

In [ ]:
def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=1):

    """
   dictionary : Gensim dictionary
   corpus : Gensim corpus
   texts : Our corpus
   limit : Maximum number of topics
   model_list : The list of LDA models
   coherence_values : Coherencies corresponding to LDA model
    """

    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        print(num_topics)
        model = LdaModel(corpus=corpus, id2word=dictionary, num_topics=num_topics)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v', window_size=300)
        coherence_values.append(coherencemodel.get_coherence())
    return model_list, coherence_values

#### Подкорпус текстов на английском языке

In [ ]:
def prep_data(texts, no_below=15, no_above=0.90):
    dictionary = Dictionary(texts)
    dictionary.filter_extremes(no_below=no_below, no_above=no_above)

    corpus = [dictionary.doc2bow(doc) for doc in texts]
    tfidf = models.TfidfModel(corpus)
    corpus_tfidf = tfidf[corpus]
    print(f'Число уникальных токенов (dictionary): {len(dictionary)}')
    print(f'Число докуменов (corpora): {len(corpus)}')

    return dictionary, corpus, corpus_tfidf

In [ ]:
dictionary_en, corpus_en, corpus_tfidf_en = prep_data(data_en)

In [ ]:
# Recall the function
model_list_en, coherence_values_en = compute_coherence_values(dictionary=dictionary_en, corpus=corpus_tfidf_en,texts=data_en, start=5, \
                                                        limit=25, step=1)

In [ ]:

limit=25; start=5; step=1;
x = range(start, limit, step)
plt.plot(x, coherence_values_en)
plt.xlabel("Количество тем")
plt.ylabel("Согласованность")
plt.legend(("coherence_values"), loc='best')
plt.suptitle("Качество модели LDA на английском подкорпусе")
plt.show()

In [ ]:
lda_model_en = models.LdaModel(corpus=corpus_tfidf_en,
                               id2word=dictionary_en,
                               num_topics=15)

lda_model_en.print_topics(num_topics=-1, num_words=17)

In [ ]:
my_dict_en = {'Topic_' + str(i): [token for token, score in lda_model_en.show_topic(i, topn=15)] for i in range(0, lda_model_en.num_topics)}

#### Подкорпус текстов на русском языке

In [ ]:
dictionary_ru, corpus_ru, corpus_tfidf_ru = prep_data(data_ru)

In [ ]:
model_list_ru, coherence_values_ru = compute_coherence_values(dictionary=dictionary_ru, corpus=corpus_ru, texts=data_ru, start=5, \
                                                        limit=25, step=1)

In [ ]:
limit=25; start=5; step=1;
x = range(start, limit, step)
plt.plot(x, coherence_values_ru)
plt.xlabel("Количество тем")
plt.ylabel("Согласованность")
plt.legend(("coherence_values"), loc='best')
#plt.suptitle("Качество модели LDA на русском подкорпусе")
plt.show()

In [ ]:
lda_model_ru = models.LdaModel(corpus=corpus_tfidf_ru,
                               id2word=dictionary_ru,
                               num_topics=15)



In [ ]:
lda_model_ru.print_topics(num_topics=-1, num_words=17)

In [ ]:
my_dict_ru = {'Topic_' + str(i): [token for token, score in lda_model_ru.show_topic(i, topn=15)] for i in range(0, lda_model_ru.num_topics)}

In [ ]:
my_dict_ru


## Стандартная модель LDA для общего корпуса

In [ ]:
data_all = data_en + data_ru
print(len(data_all))

In [ ]:
# на этом этапе каждому слову дается уникальный цифровой номер
dictionary_all, corpus_all, corpus_tfidf_all = prep_data(data_all)

In [ ]:
model_list_all, coherence_values_all = compute_coherence_values(dictionary=dictionary_all, corpus=corpus_tfidf_all, texts=data_all, start=10, \
                                                        limit=45, step=1)

In [ ]:
limit=45; start=10; step=1;
x = range(start, limit, step)
plt.plot(x, coherence_values_all)
plt.xlabel("Количество тем")
plt.ylabel("Согласованность")
plt.legend(("coherence_values_all"), loc='best')
# plt.suptitle("Качество модели LDA на общем корпусе")
plt.show()

In [ ]:
lda_model_all = models.LdaModel(corpus=corpus_tfidf_all,
                               id2word=dictionary_all,
                               num_topics=30)

coherence_model_lda = CoherenceModel(model=lda_model_all, texts=data_all, dictionary=dictionary_all, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

In [ ]:
lda_model_all.print_topics(num_topics=-1, num_words=18)

In [ ]:
my_dict_all_2 = {'Topic_' + str(i): [token for token, score in lda_model_all.show_topic(i, topn=15)] for i in range(0, lda_model_all.num_topics)}

## Корпус объеденных документов


### Ненормализорованные 

In [ ]:
nonnormalized = []
for i in range(3211):
    nonnormalized.append(data_en[i] + data_ru[i])
    

In [ ]:
len(nonnormalized)

In [ ]:
# на этом этапе каждому слову дается уникальный цифровой номер
dictionary_nonnorm = Dictionary(nonnormalized)

# убираем солова, которые встречаются слишком редко (указывая минимальное число документов) 
# или слишком часто (указывая процент документов)
dictionary_nonnorm.filter_extremes(no_below=15)

# преобразуем документы в мешок слов
corpus_nonnorm = [dictionary_nonnorm.doc2bow(doc) for doc in nonnormalized]

tfidf_nonnorm = models.TfidfModel(corpus_nonnorm)
corpus_tfidf_nonnorm = tfidf_nonnorm[corpus_nonnorm]

print(f'Число уникальных токенов (dictionary_nonnorm): {len(dictionary_nonnorm)}')
print(f'Число докуменов (corpora_nonnorm): {len(corpus_nonnorm)}')

In [ ]:
limit=25; start=5; step=1;
x = range(start, limit, step)
plt.plot(x, coherence_values_nonnorm)
plt.xlabel("Количество тем")
plt.ylabel("Согласованность")
plt.legend(("coherence_values_nonnorm"), loc='best')
# plt.suptitle("Качество модели LDA на общем корпусе")
plt.show()

In [ ]:
lda_model_nn = models.LdaModel(corpus=corpus_tfidf_nonnorm,
                               id2word=dictionary_nonnorm,
                               num_topics=18)
coherence_model_lda = CoherenceModel(model=lda_model_nn, texts=nonnormalized, dictionary=dictionary_nonnorm, coherence='c_v', 
                                    window_size=1000)
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

In [ ]:
my_dict_nn = {'Topic_' + str(i): [token for token, score in lda_model_nn.show_topic(i, topn=15)] for i in range(0, lda_model_nn.num_topics)}

In [ ]:
my_dict_nn

### Нормализованные длины

In [ ]:
import random 
normalized = []
for i in range(3211):
    random.shuffle(data_en[i])
    random.shuffle(data_ru[i])
    normalized.append(data_en[i][:150] + data_ru[i][:150])

In [ ]:
dictionary_norm = Dictionary(normalized)

# убираем солова, которые встречаются слишком редко (указывая минимальное число документов) 
# или слишком часто (указывая процент документов)
dictionary_norm.filter_extremes(no_below=15)

# преобразуем документы в мешок слов
corpus_norm = [dictionary_norm.doc2bow(doc) for doc in normalized]

tfidf_norm = models.TfidfModel(corpus_norm)
corpus_tfidf_norm = tfidf_norm[corpus_norm]

print(f'Число уникальных токенов (dictionary_norm): {len(dictionary_norm)}')
print(f'Число докуменов (corpora_norm): {len(corpus_norm)}')

In [ ]:
model_list_norm, coherence_values_norm = compute_coherence_values(dictionary=dictionary_norm, corpus=corpus_tfidf_norm, texts=normalized, start=5, \
                                                        limit=25, step=1)

In [ ]:
limit=22; start=5; step=1;
x = range(start, limit, step)
plt.plot(x, coherence_values_norm[3:])
plt.xlabel("Количество тем")
plt.ylabel("Согласованность")
plt.legend(("coherence_values_norm"), loc='best')
# plt.suptitle("Качество модели LDA на общем корпусе")
plt.show()